In [12]:
import os
import re
import mikeio
import numpy as np
import pandas as pd
import datetime as dt
from res1d_v5 import *
import pickle
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
from IPython.display import clear_output

Brainstorm

Add flood zoomable floodmap
Add 1s result timestep for pump stations


In [13]:
#General variables

period_specs = []
period_specs.append('DWF')
for i in range(1,6):
    period_specs.append('WWF' + str(i))
period_specs

['DWF', 'WWF1', 'WWF2', 'WWF3', 'WWF4', 'WWF5']

In [202]:
#Model specific variables
model_area = "NSSA"

rolling_average = False

#Only for debug then remove
gauges = ['HL14']

summation_csv = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\Summation.csv"
calibration_sheet = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Calibration_Master.xlsm"
rainfall_dfs0_file = r"J:\SEWER_AREA_MODELS\NSSA\03_SIMULATION_WORK\SYSTEM_ASSESSMENT\DATA\RAINFALL\NSSA_Rainfall_Data_PDT.dfs0"

map_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Report_Maps"
output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports"
result_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\06. CALIB_RESULTS\Latest_Backup49"

dfs0_folders = []
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS")
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\PS")

,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
Gauge,,,,,,,,,,,,,,,,,
FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node


In [313]:
all_types = []
for res_type in res_types:
    all_types.append([res_type,'Population','pe','m<sup>3</sup>/pe/d'])
for ici_type in ici_types:
    all_types.append([ici_type,'Area','ha','m<sup>3</sup>/ha/d'])
all_types.append(['Baseflow','','',''])
all_types.append(['Total','','',''])
all_types = pd.DataFrame(all_types, columns =['Load_Type','Description','Unit1','Unit2'])
all_types.set_index('Load_Type',inplace=True)
all_types

,Description,Unit1,Unit2
Load_Type,,,
Mixed,Population,pe,m<sup>3</sup>/pe/d
ResHD,Population,pe,m<sup>3</sup>/pe/d
ResLD,Population,pe,m<sup>3</sup>/pe/d
Commercial,Area,ha,m<sup>3</sup>/ha/d
Industrial,Area,ha,m<sup>3</sup>/ha/d
Institutional,Area,ha,m<sup>3</sup>/ha/d
Baseflow,,,
Total,,,


In [311]:
res_types = ['Mixed','ResHD','ResLD']
ici_types = ['Commercial','Industrial','Institutional']
ww_types = res_types + ici_types


dwf_specs = pd.read_excel(calibration_sheet,sheet_name="Parameters_DWF",skiprows=1)
for col in dwf_specs.columns:
    if col == 'Location':
        dwf_specs.rename(columns={col:'Zone'},inplace=True)
    if col in res_types or col in ici_types:
        dwf_specs.rename(columns={col:col+'_Rate'},inplace=True)
    if col[-2:] == '.1' and col[:-2] in res_types:
        dwf_specs.rename(columns={col:col[:-2]+'_Population'},inplace=True)
    if col[-2:] == '.1' and col[:-2] in ici_types:
        dwf_specs.rename(columns={col:col[:-2]+'_Area'},inplace=True)
    if col[-2:] == '.2':
        dwf_specs.rename(columns={col:col[:-2]+'_WaterLoad'},inplace=True)
    if col == 'Total' or col == 'Baseflow':
        dwf_specs.rename(columns={col:col+'_WaterLoad'},inplace=True)
    if col[-2:] == '.3' and col[:-2] in res_types:
        dwf_specs.rename(columns={col:col[:-2]+'_Population_Upstream'},inplace=True)
    if col[-2:] == '.3' and col[:-2] in ici_types:
        dwf_specs.rename(columns={col:col[:-2]+'_Area_Upstream'},inplace=True)
    if col[-2:] == '.4' or col == 'Total.1' or col == 'Baseflow.1':
        dwf_specs.rename(columns={col:col[:-2]+'_WaterLoad_Upstream'},inplace=True)
#     if col == 'Baseflow.1':
#         dwf_specs.rename(columns={col:col[:-2]+'_Upstream'},inplace=True)
dwf_specs.set_index('Zone',inplace=True)
dwf_specs.head(2)

,Mixed_Rate,ResHD_Rate,ResLD_Rate,Commercial_Rate,Industrial_Rate,Institutional_Rate,Mixed_Population,ResHD_Population,ResLD_Population,Commercial_Area,...,Industrial_Area_Upstream,Institutional_Area_Upstream,Mixed_WaterLoad_Upstream,ResHD_WaterLoad_Upstream,ResLD_WaterLoad_Upstream,Commercial_WaterLoad_Upstream,Industrial_WaterLoad_Upstream,Institutional_WaterLoad_Upstream,Baseflow_WaterLoad_Upstream,Total_WaterLoad_Upstream
Zone,,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FA2,0.185,0.185,0.185,10.0,10.0,10.0,124.697553,1376.568852,19538.388103,4.578282,...,0.0,14.067279,23.069047,254.665238,3614.601799,45.782818,0.0,140.672793,2557.439,6636.230695


In [312]:
dwf_specs.columns

Index(['Mixed_Rate', 'ResHD_Rate', 'ResLD_Rate', 'Commercial_Rate',
       'Industrial_Rate', 'Institutional_Rate', 'Mixed_Population',
       'ResHD_Population', 'ResLD_Population', 'Commercial_Area',
       'Industrial_Area', 'Institutional_Area', 'Mixed_WaterLoad',
       'ResHD_WaterLoad', 'ResLD_WaterLoad', 'Commercial_WaterLoad',
       'Industrial_WaterLoad', 'Institutional_WaterLoad', 'Baseflow_WaterLoad',
       'Total_WaterLoad', 'Mixed_Population_Upstream',
       'ResHD_Population_Upstream', 'ResLD_Population_Upstream',
       'Commercial_Area_Upstream', 'Industrial_Area_Upstream',
       'Institutional_Area_Upstream', 'Mixed_WaterLoad_Upstream',
       'ResHD_WaterLoad_Upstream', 'ResLD_WaterLoad_Upstream',
       'Commercial_WaterLoad_Upstream', 'Industrial_WaterLoad_Upstream',
       'Institutional_WaterLoad_Upstream', 'Baseflow_WaterLoad_Upstream',
       'Total_WaterLoad_Upstream'],
      dtype='object')

In [366]:
gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
gauges = gauges.loc[:, :'Level Type']
gauges = gauges[gauges.Model.str.contains(model_area)]
gauges.set_index('Gauge',inplace=True)

periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
periods = periods[periods.Model.str.contains(model_area)]

summation_df = pd.read_csv(summation_csv,dtype={'MUID': str,'SUMTO': str})

report_text = pd.read_excel(calibration_sheet,sheet_name="Report_Text")
report_text = report_text[report_text.Model.notna()]
report_text = report_text[report_text.Model.str.contains(model_area)]

wwf_specs = pd.read_excel(calibration_sheet,sheet_name="Parameters_WWF")
wwf_specs.set_index('Location',inplace=True)
wwf_stats_specs = pd.read_csv('WWF_Stats_Specs.csv')

wwf_stats_specs

,Description,DF,DF_Col,Unit,Table
0,Acc. rainfall,NaN,NaN,mm,1
1,Drainage area,wwf_specs,Drainage area (ha),ha,1
2,Number sanitary,wwf_specs,Number sanitary,-,1
3,Number stormwater,wwf_specs,Number combined,-,1
4,Number combined,wwf_specs,Number stormwater,-,1
5,Area sanitary,wwf_specs,Area sanitary (ha),ha,1
6,Area combined,wwf_specs,Area combined (ha),ha,1
7,Area stormwater,wwf_specs,Area stormwater (ha),ha,1
8,Flat impervious area,wwf_specs,Flat Impervious area (ha),ha,2
9,Steep imp. area,wwf_specs,Steep Impervious area (ha),ha,2


In [89]:
gauges.head(50)


,Gauge,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
223,FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
224,GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
225,GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
226,GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
227,GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
228,GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
229,GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
230,HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
231,HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node
232,HL18,49559,NaN,NaN,HL14,9708,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.55,MH18,Measured,Node


In [17]:
level_pipes = gauges[gauges['Level Type'].str.contains('Link')][['Node1 (Or Pipe if pipe level)','Level Type']].copy()
level_pipes.rename(columns={'Node1 (Or Pipe if pipe level)':'Node'},inplace=True)
level_pipes

,Node,Level Type
241,43987,Link(DS)
253,54852,Link(DS)
254,54850,Link(DS)


In [18]:
nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
level_nodes = nodes1 + nodes2
#Convert all to string
level_nodes = [str(x) for x in level_nodes]
#Make set to clear duplicates
level_nodes = set(level_nodes)

ds_level_pipes = gauges[gauges['Level Type']=='Link(DS)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
us_level_pipes = gauges[gauges['Level Type']=='Link(US)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
flow_pipes = gauges.Pipe.dropna().unique().flatten().tolist()

#Convert all to string
ds_level_pipes = [str(x) for x in ds_level_pipes]
us_level_pipes = [str(x) for x in us_level_pipes]
flow_pipes = [str(x) for x in flow_pipes]

#Make set to clear duplicates
ds_level_pipes = set(ds_level_pipes)
us_level_pipes = set(us_level_pipes)
flow_pipes = set(flow_pipes)

In [54]:
for ds_level_pipe in ds_level_pipes:
    print(ds_level_pipe)

54852
43987
54850


In [20]:
from mikeio.dfs0 import Dfs0
dfs0_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS"

dfs = Dfs0()
res = mikeio.read(dfs0_folder + '\\' 'HL14_20180614_20190403_5min.dfs0')
df = res.to_dataframe()
df.iloc[:,[0,6]]

,HL14_Flow_L,HL14_Level
2018-06-14 00:00:00,105.150002,-0.659200
2018-06-14 00:05:00,106.050003,-0.657905
2018-06-14 00:10:00,108.650002,-0.654010
2018-06-14 00:15:00,110.349998,-0.650690
2018-06-14 00:20:00,110.150002,-0.650770
...,...,...
2019-04-03 23:35:00,109.550003,-0.649750
2019-04-03 23:40:00,110.150002,-0.648520
2019-04-03 23:45:00,109.849998,-0.648040
2019-04-03 23:50:00,108.699997,-0.649165


In [21]:
f = 'HL14_20180614_20190403_5min.dfs0'

ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
gauge_id = re.split('_|.',f)[0]
ts['Gauge'] = gauge_id
ts.rename(columns={ts.columns[1]:'Flow'},inplace=True)
if len(ts.columns)>6:
    ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
else:
    ts['Level'] = np.nan
ts = ts[['Gauge','Flow','Level']]


In [22]:
#Import rain gauges

rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()

#Import flow/level gauges
gauge_ids = []
first_dfs0 = True
for dfs0_folder in dfs0_folders:
    for f in os.listdir(dfs0_folder):
        if f[-5:]=='.dfs0':
            gauge_id = re.split(r'_|[.]',f)[0]
            ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
            ts['Gauge'] = gauge_id
            gauge_ids.append(gauge_id)
            ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
            if len(ts.columns)>5:
                ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
            else:
                ts['Level'] = np.nan
            ts = ts[['Gauge','Flow','Level']]
            if first_dfs0 == True:
                measured = ts.copy()
            else:
                measured = pd.concat([measured,ts])
            first_dfs0 = False
            
        

In [23]:
# nodes = resultdata.resultData.Nodes
# for i, node in enumerate(nodes):
#     rid = node.Id


#     if rid in level_nodes:
#         print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
#         for di in node.DataItems:
#             if di.Quantity.Id.lower() == 'waterlevel':



IndentationError: expected an indented block (2035513836.py, line 11)

In [62]:
list(di.CreateTimeSeriesData(2))

[12.31151,
 12.31077,
 12.31001,
 12.30925,
 12.30849,
 12.30771,
 12.30694,
 12.30617,
 12.30546,
 12.30481,
 12.30417,
 12.30355,
 12.30291,
 12.30227,
 12.30163,
 12.30099,
 12.30033,
 12.29967,
 12.29901,
 12.29837,
 12.29785,
 12.29749,
 12.29718,
 12.29689,
 12.2966,
 12.29632,
 12.29603,
 12.29574,
 12.29545,
 12.29515,
 12.29486,
 12.29457,
 12.29432,
 12.29412,
 12.29393,
 12.29374,
 12.29356,
 12.29338,
 12.29319,
 12.293,
 12.29281,
 12.29262,
 12.29244,
 12.29226,
 12.29214,
 12.29211,
 12.2921,
 12.29211,
 12.29212,
 12.29212,
 12.29213,
 12.29214,
 12.29215,
 12.29216,
 12.29217,
 12.29219,
 12.29225,
 12.29237,
 12.29251,
 12.29266,
 12.29282,
 12.29297,
 12.29313,
 12.29328,
 12.29344,
 12.29359,
 12.29374,
 12.29393,
 12.2944,
 12.29522,
 12.29615,
 12.29709,
 12.29806,
 12.29901,
 12.29994,
 12.30088,
 12.3018,
 12.3027,
 12.30361,
 12.30452,
 12.30559,
 12.30682,
 12.30808,
 12.30933,
 12.31058,
 12.31182,
 12.31304,
 12.31424,
 12.31543,
 12.31661,
 12.31778,
 12.31

In [194]:
results = []
first_level = True
first_flow = True

for f in os.listdir(result_folder):
    if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f:
        resultdata = Res1D_v5(result_folder + '\\' + f)
        reaches = resultdata.resultData.Reaches
        nodes = resultdata.resultData.Nodes
        
        print ("Importing network " + f + " at " + str(dt.datetime.now()))

#         missing_muids = set()

#         #Check for missing MUIDs in the result file
#         muid_checks = set()

#         rid_checks = set()
#         for i, reach in enumerate(reaches):
#             rid_checks.add(reach.Id[:(-1 * len(str(i)) - 1)])
#         for rid in network_import_specs["MUID_Extended"].values:
#             if not rid in rid_checks:
#                 muid_checks.add(rid)
#         if len(muid_checks)>0:
#             print('In Network_Import_Specs but not in re: ' + str(muid_checks))
#             break

        for i, node in enumerate(nodes):
            
            rid = node.Id
            if rid in level_nodes:
                print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
                for di in node.DataItems:
                    if di.Quantity.Id.lower() == 'waterlevel':

                        values = list(di.CreateTimeSeriesData(0))
                        level_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        level_df['ResultFile'] = f
                        level_df['MUID'] = rid
                        level_df['Level'] = values                          

                        level_df['DateTimeRef'] = level_df.index

                        if first_level == True:
                            level_df_all = level_df.copy()
                            
                        else:
                            level_df_all = pd.concat([level_df_all,level_df])
                        first_level = False
        
        
        first_round = True
        for i, reach in enumerate(reaches):
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
            gridpoints = reach.GridPoints.Count
            levelpoints = (gridpoints + 1)/2

            if rid in us_level_pipes or rid in ds_level_pipes:
            
                print ("Importing pipe " + reach.Id + " level at " + str(dt.datetime.now()))
                print (gridpoints)
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'waterlevel':

                        if rid in us_level_pipes: 
                            values = list(di.CreateTimeSeriesData(0))
                        else:
                            values = list(di.CreateTimeSeriesData(levelpoints - 1))
                            
                        
                        level_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        level_df['ResultFile'] = f
#                         flow_df['MUID'] = network_import_specs[network_import_specs['MUID_Extended']==rid].iloc[0]['MUID']
                        level_df['MUID'] = rid
                        level_df['Level'] = values                          

                        level_df['DateTimeRef'] = level_df.index

                        if first_level == True:
                            level_df_all = level_df.copy()
                            
                        else:
                            level_df_all = pd.concat([level_df_all,level_df])
                        first_level = False
            
        for i, reach in enumerate(reaches):
            
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
            gridpoints = reach.GridPoints.Count
            if rid in flow_pipes or rid in list(summation_df.MUID):
                print ("Importing pipe " + reach.Id + " discharge at " + str(dt.datetime.now()))
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'discharge':

                        values = list(di.CreateTimeSeriesData(0))
                        flow_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        flow_df['ResultFile'] = f
                        flow_df['MUID'] = rid
                        flow_df['DateTimeRef'] = flow_df.index
                        flow_df['Discharge'] = values                          
                        flow_df['Discharge'] = flow_df['Discharge'] * 1000

                        if rolling_average == True:
                            flow_df['Discharge_Hourly'] = flow_df['Discharge'].rolling('1h').mean()

                        

                        if first_flow == True:
                            flow_df_all = flow_df.copy()

                        else:
                            flow_df_all = pd.concat([flow_df_all,flow_df])
                        first_flow = False

        first_round = False
        
#Summation
df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum'})
df_result_sum.reset_index(inplace=True)
df_result_sum.set_index('DateTimeRef',drop=False,inplace=True)
df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)
flow_df_all = pd.concat([flow_df_all,df_result_sum])


Importing network NSSA_DWF_Cal_2018-07-26_Base.res1d at 2022-10-27 07:46:00.484811
Importing node 10497 at 2022-10-27 07:46:00.484811
Importing node 10498 at 2022-10-27 07:46:00.484811
Importing node 1_119 at 2022-10-27 07:46:00.500436
Importing node 6071 at 2022-10-27 07:46:00.500436
Importing node 6124 at 2022-10-27 07:46:00.500436
Importing node 6819 at 2022-10-27 07:46:00.516062
Importing node 6822 at 2022-10-27 07:46:00.516062
Importing node 7133 at 2022-10-27 07:46:00.516062
Importing node 7169 at 2022-10-27 07:46:00.516062
Importing node 7317 at 2022-10-27 07:46:00.531687
Importing node 9488 at 2022-10-27 07:46:00.531687
Importing node 9525 at 2022-10-27 07:46:00.531687
Importing node 9551 at 2022-10-27 07:46:00.547312
Importing node 9561 at 2022-10-27 07:46:00.547312
Importing node 9600 at 2022-10-27 07:46:00.547312
Importing node 9619 at 2022-10-27 07:46:00.547312
Importing node 9624 at 2022-10-27 07:46:00.562938
Importing node 9629 at 2022-10-27 07:46:00.562938
Importing node

Importing network NSSA_DWF_Cal_2018-08-25_Base.res1d at 2022-10-27 07:46:02.673160
Importing node 10497 at 2022-10-27 07:46:02.673160
Importing node 10498 at 2022-10-27 07:46:02.673160
Importing node 1_119 at 2022-10-27 07:46:02.688786
Importing node 6071 at 2022-10-27 07:46:02.688786
Importing node 6124 at 2022-10-27 07:46:02.688786
Importing node 6819 at 2022-10-27 07:46:02.704410
Importing node 6822 at 2022-10-27 07:46:02.704410
Importing node 7133 at 2022-10-27 07:46:02.704410
Importing node 7169 at 2022-10-27 07:46:02.720036
Importing node 7317 at 2022-10-27 07:46:02.720036
Importing node 9488 at 2022-10-27 07:46:02.735662
Importing node 9525 at 2022-10-27 07:46:02.735662
Importing node 9551 at 2022-10-27 07:46:02.735662
Importing node 9561 at 2022-10-27 07:46:02.735662
Importing node 9600 at 2022-10-27 07:46:02.751286
Importing node 9619 at 2022-10-27 07:46:02.751286
Importing node 9624 at 2022-10-27 07:46:02.751286
Importing node 9629 at 2022-10-27 07:46:02.766912
Importing node

Importing pipe 54852-1539 discharge at 2022-10-27 07:46:04.137224
Importing pipe 55199-1545 discharge at 2022-10-27 07:46:04.143343
Importing pipe 56296-1593 discharge at 2022-10-27 07:46:04.151348
Importing pipe 56393-1597 discharge at 2022-10-27 07:46:04.157353
Importing pipe 56394_3-1600 discharge at 2022-10-27 07:46:04.162357
Importing pipe 56401-1605 discharge at 2022-10-27 07:46:04.169364
Importing pipe 56405-1606 discharge at 2022-10-27 07:46:04.176370
Importing pipe Link_15-1612 discharge at 2022-10-27 07:46:04.181376
Importing network NSSA_WWF_Cal_2018-11-25_Base.res1d at 2022-10-27 07:46:04.774065
Importing node 10497 at 2022-10-27 07:46:04.774065
Importing node 10498 at 2022-10-27 07:46:04.774065
Importing node 1_119 at 2022-10-27 07:46:04.789691
Importing node 6071 at 2022-10-27 07:46:04.789691
Importing node 6124 at 2022-10-27 07:46:04.805315
Importing node 6819 at 2022-10-27 07:46:04.805315
Importing node 6822 at 2022-10-27 07:46:04.805315
Importing node 7133 at 2022-10-2

Importing pipe 49709-1018 discharge at 2022-10-27 07:46:06.293052
Importing pipe 49727-1036 discharge at 2022-10-27 07:46:06.299055
Importing pipe 50870-1397 discharge at 2022-10-27 07:46:06.310065
Importing pipe 50924-1435 discharge at 2022-10-27 07:46:06.317071
Importing pipe 50932-1443 discharge at 2022-10-27 07:46:06.325079
Importing pipe 50933-1444 discharge at 2022-10-27 07:46:06.332085
Importing pipe 50940-1451 discharge at 2022-10-27 07:46:06.340093
Importing pipe 50952-1463 discharge at 2022-10-27 07:46:06.348134
Importing pipe 54280-1513 discharge at 2022-10-27 07:46:06.348134
Importing pipe 54287-1519 discharge at 2022-10-27 07:46:06.348134
Importing pipe 54850-1537 discharge at 2022-10-27 07:46:06.363758
Importing pipe 54852-1539 discharge at 2022-10-27 07:46:06.379383
Importing pipe 55199-1545 discharge at 2022-10-27 07:46:06.379383
Importing pipe 56296-1593 discharge at 2022-10-27 07:46:06.379383
Importing pipe 56393-1597 discharge at 2022-10-27 07:46:06.395009
Importing 

Importing pipe 47438-627 discharge at 2022-10-27 07:46:08.730109
Importing pipe 49559-884 discharge at 2022-10-27 07:46:08.742120
Importing pipe 49580-901 discharge at 2022-10-27 07:46:08.750127
Importing pipe 49582-903 discharge at 2022-10-27 07:46:08.759136
Importing pipe 49592-913 discharge at 2022-10-27 07:46:08.766142
Importing pipe 49640-950 discharge at 2022-10-27 07:46:08.777152
Importing pipe 49652-962 discharge at 2022-10-27 07:46:08.781196
Importing pipe 49709-1018 discharge at 2022-10-27 07:46:08.781196
Importing pipe 49727-1036 discharge at 2022-10-27 07:46:08.796821
Importing pipe 50870-1397 discharge at 2022-10-27 07:46:08.812446
Importing pipe 50924-1435 discharge at 2022-10-27 07:46:08.812446
Importing pipe 50932-1443 discharge at 2022-10-27 07:46:08.828072
Importing pipe 50933-1444 discharge at 2022-10-27 07:46:08.828072
Importing pipe 50940-1451 discharge at 2022-10-27 07:46:08.843696
Importing pipe 50952-1463 discharge at 2022-10-27 07:46:08.859322
Importing pipe 54

In [191]:
df_result_sum.set_index('DateTimeRef',drop=False)

,ResultFile,MUID,DateTimeRef,Discharge
DateTimeRef,,,,
2018-07-26 00:00:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:00:00,184.971800
2018-07-26 00:05:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:05:00,0.000000
2018-07-26 00:10:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:10:00,0.000000
2018-07-26 00:15:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:15:00,0.000000
2018-07-26 00:20:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:20:00,186.896800
...,...,...,...,...
2020-02-01 23:40:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:40:00,127.503012
2020-02-01 23:45:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:45:00,127.694494
2020-02-01 23:50:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:50:00,126.869387


In [193]:
df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum'})
df_result_sum.reset_index(inplace=True)
df_result_sum.set_index('DateTimeRef',drop=False,inplace=True)
df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)
df_result_sum

,ResultFile,MUID,DateTimeRef,Discharge
DateTimeRef,,,,
2018-07-26 00:00:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:00:00,184.971800
2018-07-26 00:05:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:05:00,0.000000
2018-07-26 00:10:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:10:00,0.000000
2018-07-26 00:15:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:15:00,0.000000
2018-07-26 00:20:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:20:00,186.896800
...,...,...,...,...
2020-02-01 23:40:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:40:00,127.503012
2020-02-01 23:45:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:45:00,127.694494
2020-02-01 23:50:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:50:00,126.869387


In [183]:
df_result_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18450 entries, 0 to 18449
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ResultFile   18450 non-null  object        
 1   MUID         18450 non-null  object        
 2   DateTimeRef  18450 non-null  datetime64[ns]
 3   Discharge    18450 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 576.7+ KB


In [184]:
flow_df_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 369000 entries, 2018-07-26 00:00:00 to 2020-02-02 00:00:00
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   ResultFile   369000 non-null  object        
 1   MUID         369000 non-null  object        
 2   Discharge    369000 non-null  float64       
 3   DateTimeRef  369000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 14.1+ MB


In [ ]:
# import plotly
# a = np.random.normal(0,1,100)
# b = np.random.normal(-2,5,100)

# c = np.random.normal(0,1,100)
# d = np.random.normal(-2,5,100)

# fig = plotly.tools.make_subplots(rows=2,cols=1)

# trace_rm1 = go.Histogram(x = a, opacity = 0.75, name = 'malignant')
# trace_rm2 = go.Histogram(x = b, opacity = 0.75, name = 'benign')
# fig.append_trace(go.Histogram(x = a, opacity = 0.75, name = 'benign'),1,1)
# fig.append_trace(go.Histogram(x = b, opacity = 0.75, name = 'malignant'),1,1)
# fig.append_trace(go.Histogram(x = c, opacity = 0.75, name = 'benign'),2,1)
# fig.append_trace(go.Histogram(x = d, opacity = 0.75, name = 'malignant'),2,1)
# fig.layout.update(go.Layout(barmode = 'overlay',))

# fig.show()

In [112]:
flow_df_all

,ResultFile,MUID,Discharge,DateTimeRef
2018-07-26 00:00:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,16.468840,2018-07-26 00:00:00
2018-07-26 00:05:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,16.175270,2018-07-26 00:05:00
2018-07-26 00:10:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.881900,2018-07-26 00:10:00
2018-07-26 00:15:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.589100,2018-07-26 00:15:00
2018-07-26 00:20:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.296860,2018-07-26 00:20:00
...,...,...,...,...
18445,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,127.503012,2020-02-01 23:40:00
18446,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,127.694494,2020-02-01 23:45:00
18447,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,126.869387,2020-02-01 23:50:00
18448,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,125.155301,2020-02-01 23:55:00


In [188]:
flow_df_all

,ResultFile,MUID,DateTimeRef,Discharge
2018-07-26 00:00:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:00:00,16.468840
2018-07-26 00:05:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:05:00,16.175270
2018-07-26 00:10:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:10:00,15.881900
2018-07-26 00:15:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:15:00,15.589100
2018-07-26 00:20:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:20:00,15.296860
...,...,...,...,...
18445,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:40:00,127.503012
18446,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:45:00,127.694494
18447,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:50:00,126.869387
18448,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:55:00,125.155301


In [153]:
gauges.columns

Index(['Pipe', 'CSO1', 'CSO2', 'Location', 'Node1 (Or Pipe if pipe level)',
       'Node2', 'CSOPipe1', 'CSOPipe2', 'Rain Gauge', 'Model', 'Sensor Output',
       'Location Type', 'Pipe Shape', 'Pipe Dimension', 'Node1 AssetName',
       'Flow source', 'Level Type'],
      dtype='object')

In [265]:
error_list = []
for gauge in gauge_ids[:1]:
    
#     try:

        
        zone = str(gauges.loc[gauge,'Location'])
        pipe = str(gauges.loc[gauge,'Pipe'])
        node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
        rain_gauge = str(gauges.loc[gauge,'Rain Gauge'])
        MonitoringType = str(gauges.loc[gauge,"Sensor Output"])
        NodeID = str(gauges.loc[gauge, "Node1 (Or Pipe if pipe level)"])
        MHName = str(gauges.loc[gauge,'Node1 AssetName'])       

        PipeShape = str(gauges.loc[gauge,'Pipe Shape'])
        PipeDimension = str(round(gauges.loc[gauge, 'Pipe Dimension'],2))


        with open(output_folder + "\\" + "Calibration_Report_" + gauge + ".html", 'w') as f:
            
            f.write('<style>')
#             f.write('@page {margin-left: 5cm;}')
            f.write('p    {width:1000px;}')
            f.write('p    { font-size: 20px;}')
            f.write('table.grid, th, td {')
            f.write('  border: 1px solid;')
            f.write('  padding: 4px;')
            f.write('  border-collapse: collapse;')
            
            f.write('})')            
            f.write('</style>')
            f.write('<h1 style="text-align:center">Model Calibration Report for gauge ' 
                    + gauge + ' in Zone ' + zone + ', ' + model_area + '</h1>')

            f.write('<h2>1. Description of the Calibration Area</h2>')
            f.write('<h2>2. Description of the Sensor and Network</h2>')            
            f.write('<table class="grid">\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Monitoring Type' +'</td>\n')
            f.write('<td>'+ MonitoringType +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Node ID' +'</td>\n')
            f.write('<td>'+ node +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe ID' +'</td>\n')
            f.write('<td>'+ pipe +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Shape' +'</td>\n')
            f.write('<td>'+ PipeShape +'</td>\n')
            f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Dimension (m) or CRS ID ' +'</td>\n')
            f.write('<td>'+ PipeDimension +'</td>\n')
            f.write('</tr>\n')
            f.write('</table>')
    
            
            f.write('<h2>3. Model Calibration Setup: </h2>')
                    
            f.write('<table class="grid">\n')
            f.write('  <tr>\n')
            f.write('    <th>Simulation</th>\n')
            f.write('    <th>Start Date</th>\n')
            f.write('    <th>End Date</th>\n')
            f.write('    <th>Duration</th>\n')
            f.write('  </tr>\n')

            for period_spec in period_specs:
                description = period_spec[:2] + ' Calibration'
                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                duration = str((end - start).days) + ' days'
                f.write('  <tr>\n')
                f.write('    <td>' + description + '</th>\n')
                f.write('    <td>' + start.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + end.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + duration + '</th>\n')
                f.write('  </tr>\n')                   
            f.write('</table>\n')
                                      
            f.write('<h2>4. Zone Map</h2>\n')
            f.write('<img src="' + map_folder + '\\' + zone + '.jpg" alt="' + gauge + '">\n')
            f.write('<h2>5. DWF Calibration Summary</h2>\n')  
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['DWF Text'])[0]) + '</p>\n')
            f.write('<h2>6. WWF Calibration and Validation Summary</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['WWF Text'])[0]) + '</p>\n')
            f.write('<h2>7. Calibration Issues</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Issue Text'])[0]) + '</p>\n')
            f.write('<h2>8. Recommendations</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Recommendation Text'])[0]) + '</p>\n')
            f.write('<h2>9. DWF Calibration Plots and Statistics</h2>\n')

            wwf_plot_header_added = False
            for period_spec in period_specs:

                if period_spec[:3] == "WWF" and wwf_plot_header_added == False:
                    f.write('<h2>10. WWF Calibration Plots</h2>\n')
                    wwf_plot_header_added = True

                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]

                chart_header = period_spec[:3] + ', ' + start.strftime("%b %d %Y") + ' - ' + end.strftime("%b %d %Y")

                measured_gauge = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)]

                measured_rain = rainfall.loc[(rainfall.index >= start) & (rainfall.index <= end)][rain_gauge].to_frame()

                result_flow_gauge = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)]

                result_level_gauge = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)]

            #     result_gauge = df_result.loc[(df_result.index >= start) & (df_result.index <= end)]

                fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=('Discharge',  'Water Level'),
                                                    specs=[[{"secondary_y": True}, {"secondary_y": True}]],
                                                   horizontal_spacing = 0.1)

                fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge, mode='lines',name='Model Flow'),1,1)   
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Flow, mode='lines',name='Measured Flow'),1,1)
                fig.add_trace(go.Scatter(x=measured_rain.index, y=measured_rain[rain_gauge], mode='lines',name='Rainfall'),1,1, secondary_y=True)

                fig.add_trace(go.Scatter(x=result_level_gauge.index, y=result_level_gauge.Level, mode='lines',name='Model Level'),1,2)
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Level, mode='lines',name='Measured Level'),1,2)

                fig.update_layout(
                    autosize=False,
                    width = 1600,
                    height=250,
                    margin=dict(
                        l=0,
                        r=0,
                        b=25,
                        t=60,
                        pad=4
                        ),
            #         yaxis_title="Dicharge (L/s)",
                    title ={
                        'text' : chart_header,
                        'x':0.45,
                        'xanchor': 'center'
                    })

                fig['layout']['yaxis']['title']='Dicharge (L/s)'
                fig['layout']['yaxis2']['title']='Rainfall (mm / 5 min)' 
            #     fig['layout']['yaxis2']['autorange']='reversed'
                fig['layout']['yaxis2']['range']=[10,0]
                fig['layout']['yaxis3']['title']='Water Level (m)'



            #     fig.update_yaxes(rangemode="tozero")    

#                 fig.show()
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))


            #             f.write('<h1>Long Term Simulation ' + w1.value + '</h1>')
            #             f.write('<h2>Model Result</h1>')

        f.close()
        print("Writing html for " + gauge)
#     except Exception as e:
#         error_list.append([gauge_id,e])

    

Writing html for FA2


In [198]:
gauge

'NV13'

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
from plotly.subplots import make_subplots
np.random.seed(123)

fig = make_subplots(specs=[[{'secondary_y': True}]])

n=8

d = {'date': [datetime(2020, 3, k ) for k in range(2, n+2)],
     'A': 21+12*np.random.rand(n),
     'B': 16+12*np.random.rand(n),
     'C': 14+10*np.random.rand(n)}
df = pd.DataFrame(d)

fig.add_trace(go.Scatter(x=df['date'], y= df['A']), secondary_y=False)  
fig.add_trace(go.Scatter(x=df['date'], y= df['B']), secondary_y=False)  

fig.add_trace(go.Scatter(x=df['date'], y= df['C']), secondary_y=True) 


In [261]:
with open(output_folder + "\\" + "Test_Stats.html", 'w') as f:
    
    
    
    f.write('<style>\n')
#             f.write('@page {margin-left: 5cm;}')
    f.write('p    {width:1000px;}\n')
    f.write('p    { font-size: 20px;}\n')
    f.write('table, th, td {\n')
    f.write('  border: 1px solid;\n')
    f.write('  padding: 4px;\n')
    f.write('  border-collapse: collapse;\n')

    f.write('})\n')            
    f.write('</style>\n')
    
    f.write('<table>\n')
    f.write('  <tr>\n')
    f.write('    <th>Simulation</th>\n')
    f.write('    <th>Start Date</th>\n')
    f.write('    <th>End Date</th>\n')
    f.write('    <th>Duration</th>\n')
    f.write('  </tr>\n')

    for period_spec in period_specs:
        description = period_spec[:2] + ' Calibration'
        start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
        end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
        duration = str((end - start).days) + ' days'
        f.write('  <tr>\n')
        f.write('    <td>' + description + '</th>\n')
        f.write('    <td>' + start.strftime("%Y-%m-%d %H:%M") + '</th>\n')
        f.write('    <td>' + end.strftime("%Y-%m-%d %H:%M") + '</th>\n')
        f.write('    <td>' + duration + '</th>\n')
        f.write('  </tr>\n')                   
    f.write('</table>\n')
    
    f.write('<table>\n')
    f.write('  <tr>\n')
    f.write('    <th>Simulation</th>\n')
    f.write('    <th>Start Date</th>\n')
    f.write('    <th>End Date</th>\n')
    f.write('    <th>Duration</th>\n')
    f.write('  </tr>\n')

    for period_spec in period_specs:
        description = period_spec[:2] + ' Calibration'
        start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
        end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
        duration = str((end - start).days) + ' days'
        f.write('  <tr>\n')
        f.write('    <td>' + description + '</th>\n')
        f.write('    <td>' + start.strftime("%Y-%m-%d %H:%M") + '</th>\n')
        f.write('    <td>' + end.strftime("%Y-%m-%d %H:%M") + '</th>\n')
        f.write('    <td>' + duration + '</th>\n')
        f.write('  </tr>\n')                   
    f.write('</table>\n')

f.close()
    

In [370]:
wwf_stats_specs = pd.read_csv('WWF_Stats_Specs.csv')
zone = 'HL14'
with open(output_folder + "\\" + "Test_Stats.html", 'w') as f:
      
    f.write('<style>\n')
#             f.write('@page {margin-left: 5cm;}')
    f.write('p    {width:1000px;}\n')
    f.write('p    { font-size: 20px;}\n')
    
    f.write('* {\n')
    f.write('  box-sizing: border-box;\n')
    f.write('}\n')

    f.write('.row {\n')
    f.write('  margin-left:-5px;\n')
    f.write('  margin-right:-5px;\n')
    
    f.write('}\n')

    f.write('.column {\n')
    f.write('  float: left;\n')
    f.write('  width: 20%;\n')
    f.write('  padding: 5px;\n')
    f.write('}\n')

    f.write('/* Clearfix (clear floats) */\n')
    f.write('.row::after {\n')
    f.write('  content: "";\n')
    f.write('  clear: both;\n')
    f.write('  display: table;\n')
    f.write('}\n')
    

    
    f.write('table.first, table.first th, table.first td {\n')
    f.write('  border: 2px solid;\n')
    f.write('  padding: 4px;\n')

    f.write('  border-collapse: collapse;\n')
    f.write('  height: 35px;\n')
    f.write('  width: 350px;\n')
    f.write('}\n') 

    f.write('table.second {\n')
    f.write('  border: 2px solid;\n')
    f.write('  height: 35px;\n')
    f.write('  width: 350px;\n')

    f.write('})\n')
    
    f.write('</style>\n')

    f.write('<div class="row">')
    f.write('<div class="column">')    
    f.write('<table class="first">\n')
    f.write('  <tr>\n')
    f.write('    <th>Load Type</th>\n')
    f.write('    <th>Loading Rate</th>\n')
    f.write('    <th>Unit</th>\n')
    f.write('  </tr>\n')

    #Loading rate
    for ww_type in ww_types:
        if ww_type in res_types:
            decimals = 3
        else: 
            decimals = 1
        rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
        unit = all_types.loc[ww_type,'Unit2']

        f.write('  <tr>\n')
        f.write('    <td>' + ww_type + '</th>\n')
        f.write('    <td>' + str(rate) + '</th>\n')
        f.write('    <td>' + unit + '</th>\n')
        f.write('  </tr>\n') 
    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
    f.write('</table>\n')
    f.write('</div>')
    
    f.write('<div class="column">')
    
    #Population and ICI areas
    f.write('<table class="first">\n')
    f.write('  <tr>\n')
    f.write('    <th>Load Type</th>\n')
    f.write('    <th>Local</th>\n')
    f.write('    <th>Total</th>\n')
    f.write('    <th>Unit</th>\n')
    f.write('  </tr>\n')
    
    for ww_type in ww_types:
        if ww_type in res_types:
            decimals = 0
        else: 
            decimals = 1
        description = unit = all_types.loc[ww_type,'Description']
        local_val = round(dwf_specs.loc[zone,ww_type + '_' + description],decimals)
        total_val = round(dwf_specs.loc[zone,ww_type + '_' + description + '_Upstream'],decimals)
        unit = all_types.loc[ww_type,'Unit1']

        f.write('  <tr>\n')
        f.write('    <td>' + ww_type + '</th>\n')
        f.write('    <td>' + str(local_val) + '</th>\n')
        f.write('    <td>' + str(total_val) + '</th>\n')
        f.write('    <td>' + unit + '</th>\n')
        f.write('  </tr>\n') 
    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
    f.write('</table>\n')
    f.write('</div>')
    
    f.write('<div class="column">')
    #Waterload
    f.write('<table class="first">\n')
    f.write('  <tr>\n')
    f.write('    <th>Load Type</th>\n')
    f.write('    <th>Local</th>\n')
    f.write('    <th>Total</th>\n')
    f.write('    <th>Unit</th>\n')
    f.write('  </tr>\n')

    for load_type in all_types.index.values.tolist():
           
        local_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad'],0)
        total_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad_Upstream'],0)
        unit = 'm<sup>3</sup>/d'

        f.write('  <tr>\n')
        f.write('    <td>' + load_type + '</th>\n')
        f.write('    <td>' + str(local_val) + '</th>\n')
        f.write('    <td>' + str(total_val) + '</th>\n')
        f.write('    <td>' + unit + '</th>\n')
        f.write('  </tr>\n') 

    f.write('</table>\n')
    f.write('</div>')
    f.write('</div>')
    
    #WWF Parameters -------------------------------------------------------------------------------
    
    total_rows = 10
    f.write('<div class="row">')
    for i in range(1,4): #Cycle through the three table types.   
        f.write('<div class="column">')    
        f.write('<table class="second">\n')
        f.write('  <tr>\n')
        f.write('    <th>Description</th>\n')
        f.write('    <th>Value</th>\n')
        f.write('    <th>Unit</th>\n')
        f.write('  </tr>\n')

        current_row = 0
        for index, row in wwf_stats_specs[wwf_stats_specs.Table==i].iterrows():

            decimals = 2

            rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
            unit = all_types.loc[ww_type,'Unit2']
            
        
            f.write('  <tr>\n')
            f.write('    <td>' + row['Description'] + '</th>\n')
            if row['DF'] == 'wwf_specs':
                f.write('    <td>' + str(round(wwf_specs.loc[zone,row['DF_Col']],2)) + '</th>\n')
            else: 
                f.write('    <td>' + 'PH' + '</th>\n')
            f.write('    <td>' + row['Unit'] + '</th>\n')
            f.write('  </tr>\n')
            current_row += 1
            
        
        remaining_rows = total_rows - current_row

        for remaining_row in range(remaining_rows):
            f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
        f.write('</table>\n')
        f.write('</div>')
    
    f.write('</div>')
       
f.close()

In [354]:
index


26

In [337]:
dir()

['Connection',
 'Dfs0',
 'In',
 'MHName',
 'MonitoringType',
 'NodeID',
 'Out',
 'PipeDimension',
 'PipeShape',
 'Res1D_v5',
 'ResultData',
 '_',
 '_100',
 '_102',
 '_105',
 '_107',
 '_108',
 '_109',
 '_110',
 '_111',
 '_112',
 '_116',
 '_117',
 '_118',
 '_119',
 '_123',
 '_13',
 '_134',
 '_135',
 '_136',
 '_137',
 '_141',
 '_144',
 '_145',
 '_147',
 '_148',
 '_15',
 '_152',
 '_153',
 '_16',
 '_17',
 '_182',
 '_187',
 '_188',
 '_189',
 '_190',
 '_191',
 '_193',
 '_197',
 '_198',
 '_199',
 '_2',
 '_20',
 '_201',
 '_203',
 '_204',
 '_205',
 '_208',
 '_209',
 '_211',
 '_212',
 '_213',
 '_214',
 '_215',
 '_216',
 '_217',
 '_219',
 '_220',
 '_221',
 '_222',
 '_223',
 '_224',
 '_225',
 '_226',
 '_227',
 '_228',
 '_233',
 '_234',
 '_237',
 '_240',
 '_241',
 '_242',
 '_243',
 '_245',
 '_246',
 '_247',
 '_248',
 '_249',
 '_250',
 '_251',
 '_253',
 '_255',
 '_256',
 '_266',
 '_267',
 '_27',
 '_270',
 '_30',
 '_305',
 '_306',
 '_307',
 '_308',
 '_309',
 '_310',
 '_311',
 '_312',
 '_313',
 '_314',

In [339]:
wwf_specs

,Imp. cal. factor ICF,Imp. san. factor ISF,Length factor LF,Slope factor SF,RDII Factor RF,Umax (mm),Lmax (mm),Overland coeff. Cqof,Groundwater coefficient GwCarea,TC overland flow (h),...,Area sanitary (ha),Area combined (ha),Area stormwater (ha),Drainage area (ha),RDII Area (ha),Steep Impervious area (ha),Flat Impervious area (ha),Total,Average Length (m),Average slope (m/m)
Location,,,,,,,,,,,,,,,,,,,,,
FA2,1,0.013,2,0.02,0.180,10,100,0.13,0.40,7,...,846.548165,0,0,846.548165,94.040395,0,4.213730,98.254126,3246.007402,0.003537
GE1,1,0.010,1,1.00,0.060,10,100,0.30,1.00,10,...,111.891809,0,0,111.891809,4.658441,0,0.340646,4.999088,1041.640768,0.213581
GE2,1,0.025,1,1.00,0.060,10,100,0.30,1.00,10,...,9.859366,0,0,9.859366,0.310570,0,0.117326,0.427896,504.680568,0.220579
GE3,1,0.010,1,1.00,0.090,10,100,0.30,1.00,10,...,124.914356,0,0,124.914356,8.144982,0,0.345752,8.490734,580.077344,0.249665
GE5,1,0.020,1,1.00,0.100,10,100,0.30,1.00,10,...,17.353165,0,0,17.353165,1.197368,0,0.107590,1.304958,814.151462,0.325905
GN14,1,0.010,1,1.00,0.070,10,100,0.30,1.00,10,...,128.361028,0,0,128.361028,6.046294,0,0.419501,6.465795,579.814265,0.309131
HL1,1,0.010,1,0.10,0.100,10,100,0.30,0.10,10,...,6.604919,0,0,6.604919,0.384406,0,0.027741,0.412147,126.684222,0.009563
HL14,1,0.015,5,0.05,0.130,10,100,0.25,0.01,6,...,801.139381,0,0,801.139381,68.478579,0,4.114419,72.592997,4193.547286,0.011822
HL33,1,0.020,1,1.00,0.100,10,100,0.30,1.00,10,...,91.862891,0,0,91.862891,6.682295,0,0.503207,7.185502,556.529619,0.332020


In [314]:
dwf_specs.columns

Index(['Mixed_Rate', 'ResHD_Rate', 'ResLD_Rate', 'Commercial_Rate',
       'Industrial_Rate', 'Institutional_Rate', 'Mixed_Population',
       'ResHD_Population', 'ResLD_Population', 'Commercial_Area',
       'Industrial_Area', 'Institutional_Area', 'Mixed_WaterLoad',
       'ResHD_WaterLoad', 'ResLD_WaterLoad', 'Commercial_WaterLoad',
       'Industrial_WaterLoad', 'Institutional_WaterLoad', 'Baseflow_WaterLoad',
       'Total_WaterLoad', 'Mixed_Population_Upstream',
       'ResHD_Population_Upstream', 'ResLD_Population_Upstream',
       'Commercial_Area_Upstream', 'Industrial_Area_Upstream',
       'Institutional_Area_Upstream', 'Mixed_WaterLoad_Upstream',
       'ResHD_WaterLoad_Upstream', 'ResLD_WaterLoad_Upstream',
       'Commercial_WaterLoad_Upstream', 'Industrial_WaterLoad_Upstream',
       'Institutional_WaterLoad_Upstream', 'Baseflow_WaterLoad_Upstream',
       'Total_WaterLoad_Upstream'],
      dtype='object')